# Diagnosis Summary Pediatrics

This notebook contain the R code to estimate the diagnosis summary aggregated by:
- site
- country
- all


First we determine the path where all the files are located and load the R libraries required. 

In [ ]:
rm(list=ls())
# Set working directory where the files are
setwd("./4CE/phase1.1/latest/")

#############
# LIBRARIES #
#############
library(dplyr)
library(tidyr)

Then we create a function to define the list with all the files to analyze.

In [ ]:
fileList <- function( path, pattern, pediatric ){
  
  fileListInput <- list.files( path = path,pattern = pattern)
  #remove old files not in use anymore
  fileListInput  <- fileListInput[! grepl( paste( c("FICHOS","VA.csv","BCH.csv","CHOP.csv", 
                                                    "RP401.csv","UAB") , collapse = "|"), x = fileListInput)]
  
  if( pediatric == TRUE){
    fileListInput  <- fileListInput[ grepl( paste( c("PED", "UNCCH"), collapse = "|"), x = fileListInput)]
    fileListInput <- fileListInput[! grepl( paste( c("APHPPED.csv"), collapse = "|"), x = fileListInput) ]
  }else{
    fileListInput  <- fileListInput[ !grepl( paste( c("PED", "BCH", "CHOP", "RP401", "UNCCH", "UAB" ), collapse = "|"), x = fileListInput)]
  }
  return( fileListInput)
}

We create another function to aggregate the diagnosis counts by country or by all. 

In [ ]:
groupData <- function( input, aggregation ){
  if( aggregation == "Country" ){
    if( select_pediatric == TRUE ){
      siteMaping <- read.delim("./phase1.1/mappingFiles/SiteID_Map_Pediatric_07-23-20.csv", colClasses = "character", sep=",")
    }else{
      siteMaping <- read.delim("./phase1.1/mappingFiles/SiteID_Map_Non_Pediatric_07-23-2020.csv", colClasses = "character", sep=",")
    }
    siteMaping <- siteMaping[,c("Acronym", "Country")]
    siteMaping$Acronym <- toupper( siteMaping$Acronym )
    input$siteid <- toupper( input$siteid)
    toanalyze <- merge( input, siteMaping, by.x = "siteid", by.y = "Acronym", all.x = TRUE )
    
    toanalyzebyCountry <- toanalyze[, -1]
    
    #icd description
    mapping <- read.delim("./phase1.1/mappingFiles/act_mapping.tsv", sep = "\t")
    mapping <- unique( mapping[, c("IcdCode", "IcdVersion", "Level2")] )
    colnames(mapping) <- c("icd", "icd_version", "description" )
    
    #we map according to the pair ICD + version to avoid issues when same code is present in ICD9 and ICD10 but refer to different diagnosis
    mapping$icdPair <- paste0( mapping$icd, "-", mapping$icd_version)
    
    toanalyzebyCountry$icdPair <- paste0( toanalyzebyCountry$icd_code_3chars, "-", toanalyzebyCountry$icd_version)
    toanalyzebyCountry <- merge( toanalyzebyCountry, mapping, by = "icdPair", all.x = TRUE)
    toanalyzebyCountry <- toanalyzebyCountry[, c(4,6,2,9)]
    byCountryOutput <- toanalyzebyCountry %>% 
      group_by(Country, description, icd_code_3chars) %>% 
      dplyr::summarise_all(~{sum(.x, na.rm = any(!is.na(.x)))})
    
    output <- as.data.frame( byCountryOutput )
  }
  if( aggregation == "All"){
    byAll <- input[, -1]
    
    #icd description
    #mapping <- read.delim("./mappingFiles/2020AA_Icd9_Icd10_Dictionary.txt", colClasses = "character")
    mapping <- read.delim("./phase1.1/mappingFiles/act_mapping.tsv", sep = "\t")
    mapping <- unique( mapping[, c("IcdCode", "IcdVersion", "Level2")] )
    colnames(mapping) <- c("icd", "icd_version", "description" )
    
    #we map according to the pair ICD + version to avoid issues when same code is present in ICD9 and ICD10 but refer to different diagnosis
    mapping$icdPair <- paste0( mapping$icd, "-", mapping$icd_version)
    
    byAll$icdPair <- paste0( byAll$icd_code_3chars, "-", byAll$icd_version)
    byAll <- merge( byAll, mapping, by = "icdPair", all.x = TRUE)
    byAll <- byAll[! is.na (byAll$icd ),]
    byAll$description_code <- paste0( byAll$description, "*", byAll$icd_code_3chars)
    
    byAll <- byAll[, c(4:5,9)]
    
    byAllOutput <- byAll %>% 
      group_by(description_code) %>% 
      dplyr::summarise_all(~{sum(.x, na.rm = any(!is.na(.x)))})
    
    output <- as.data.frame( byAllOutput )
    
  }
  return( output )
}

In [ ]:
aggregateCounts <- function( inputData, aggregationType, codeSelection = FALSE ){
  if( codeSelection != FALSE){
    inputData <- inputData[ inputData$icd_code_3chars %in% codeSelection, ]
  }
  if( aggregationType == "All"){
    toanalyze <- groupData( input = inputData, aggregation = aggregationType )
    
    toanalyze$code <- sapply(strsplit( as.character(toanalyze$description_code), "[*]"), '[', 2)
    toanalyze$description<- sapply(strsplit( as.character(toanalyze$description_code), "[*]"), '[', 1)
    
    toanalyze <- toanalyze[ order( toanalyze$num_patients_all_since_admission_diagnosis, decreasing = TRUE, na.last = TRUE), ]
    toanalyze <- toanalyze[, c(4,5,2:3)]
    toanalyze$totalPatients <- sum(as.numeric(allDemographics$totalPatients))
    toanalyze$perc_patients_all_since_admission_diagnosis <- round(100*toanalyze$num_patients_all_since_admission_diagnosis/toanalyze$totalPatients,2)
    
    
  }else if(aggregationType == "Country"){
    toanalyze <- groupData( input = inputData, aggregation = aggregationType )
    toanalyze <- toanalyze[ order( toanalyze$num_patients_all_since_admission_diagnosis, decreasing = TRUE, na.last = TRUE), ]
    
    ##add total counts and percentage per country
    if( select_pediatric == TRUE){
      siteMaping <- read.delim("./phase1.1/mappingFiles/SiteID_Map_Pediatric_07-23-20.csv", colClasses = "character", sep=",")
    }else{
      siteMaping <- read.delim("./phase1.1/mappingFiles/SiteID_Map_Non_Pediatric_07-23-2020.csv", colClasses = "character", sep=",")
    }
    siteMaping <- siteMaping[,c("Acronym", "Country")]
    siteMaping$Acronym <- toupper( siteMaping$Acronym)
    allDemographics$siteid <- toupper( allDemographics$siteid )
    demog <- merge( allDemographics, siteMaping, by.x = "siteid", by.y = "Acronym", all.x = TRUE)
    demog <- demog[,c ("Country", "totalPatients")]
    demog$totalPatients <- as.numeric( demog$totalPatients)
    demogCounts <- as.data.frame( demog %>% 
                                    group_by(Country) %>% 
                                    dplyr::summarise_all(~{sum(.x, na.rm = any(!is.na(.x)))}))
    
    toanalyze <- merge( toanalyze, demogCounts, by = "Country", all.x = TRUE)
    toanalyze$perc_patients_since_adimission_diagnosis <- round(100*toanalyze$num_patients_all_since_admission_diagnosis/toanalyze$totalPatients,2)
  }else if( aggregationType == "Site"){
    
    #icd description
    mapping <- read.delim("./phase1.1/mappingFiles/act_mapping.tsv", sep = "\t")
    mapping <- unique( mapping[, c("IcdCode", "IcdVersion", "Level2")] )
    colnames(mapping) <- c("icd", "icd_version", "description" )
    
    #we map according to the pair ICD + version to avoid issues when same code is present in ICD9 and ICD10 but refer to different diagnosis
    mapping$icdPair <- paste0( mapping$icd, "-", mapping$icd_version)
    
    inputData$icdPair <- paste0( inputData$icd_code_3chars, "-", inputData$icd_version)
    inputData <- merge( inputData, mapping, by = "icdPair", all.x = TRUE)
    
    
    if( select_pediatric == TRUE){
      siteMaping <- read.delim("./phase1.1/mappingFiles/SiteID_Map_Pediatric_07-23-20.csv", colClasses = "character", sep=",")
      inputData <- merge( inputData, siteMaping, by.x = "siteid", by.y = "Acronym", all.x = TRUE )
      toanalyze <- inputData[, c(11,9,7,5,6)]
      toanalyze$perc_patients_since_adimission_diagnosis <- round(100*toanalyze$num_patients_all_since_admission_diagnosis/toanalyze$totalPatients,2)
      
    }else{
      ## ICSM: merge together all ICSM data ##
      ICSMdata <- inputData[inputData$siteid %in% c( "ICSM1", "ICSM20", "ICSM5" ), c(-1,-2)]
      ICSM <- as.data.frame(ICSMdata %>% 
                              group_by(icd_code_3chars, icd_version.x, icd, icd_version.y, description) %>% 
                              dplyr::summarise_all(~{sum(.x, na.rm = any(!is.na(.x)))}) %>%
                              mutate( siteid = "ICSM"))
      
      #remove the individual ICSM sets and add the aggregated one
      inputData <- inputData[ ! inputData$siteid %in% c( "ICSM1", "ICSM20", "ICSM5" ), -1 ]
      inputData <- rbind( inputData, ICSM )
      
      siteMaping <- read.delim("./phase1.1/mappingFiles/SiteID_Map_Non_Pediatric_07-23-2020.csv", colClasses = "character", sep=",")
      inputData <- merge( inputData, siteMaping, by.x = "siteid", by.y = "Acronym", all.x = TRUE )
      toanalyze <- inputData[, c(11,9,7,5,6)]
      toanalyze$perc_patients_before_admission_diagnosis <- round(100*toanalyze$num_patients_all_before_admission_diagnosis/toanalyze$totalPatients,2)
      toanalyze$perc_patients_since_adimission_diagnosis <- round(100*toanalyze$num_patients_all_since_admission_diagnosis/toanalyze$totalPatients,2)
    }
    
    
  }
  return( toanalyze )
}

In [ ]:
###Create the output file when aggregating by country
excelTableFormat <- function( input, countries ){
  toanalyze <- input
  toanalyze$description_code <- paste0( toanalyze$description, "*", toanalyze$icd_code_3chars)
  for( i in 1:length(countries)){
    if(i ==1){
      cc <- toanalyze[ toanalyze$Country == countries[i], c(9, 4:8)]
      colnames(cc)[2:6] <- paste0(countries[i], "_", colnames(cc)[2:6])
    }else{
      ss <- toanalyze[ toanalyze$Country == countries[i], c(9, 4:8)]
      colnames(ss)[2:6] <- paste0(countries[i], "_", colnames(ss)[2:6])
      cc <- merge( cc, ss, all.x = TRUE, all.y = TRUE)
    }
  }
  cc$code <- sapply(strsplit( as.character(cc$description_code), "[*]"), '[', 2)
  cc$description<- sapply(strsplit( as.character(cc$description_code), "[*]"), '[', 1)
  cc <- cc[ order(cc$USA_num_patients_all_since_admission_diagnosis, decreasing = TRUE), ]
  return(cc)
}

We create the list with all the pediatric files.

In [ ]:
select_pediatric = TRUE
fileListDiag <- fileList( path = "./phase1.1/latest/",pattern = "Diag", pediatric = select_pediatric)
fileListDemog <- fileList( path = "./phase1.1/latest/",pattern = "Demog", pediatric = select_pediatric)

In [ ]:
if( select_pediatric == TRUE){
  obfuscation <- read.delim( file   = "./phase1.1_pediatric/PediatricDiagnosis/diagnosisCount_october2020/pediatric_obfuscation.txt", 
                             header = TRUE, sep = "\t")
}else{
  obfuscation <- read.delim( file   = "./phase1.1/mappingFiles/obfuscation_values.txt", 
                             header = TRUE, sep = "\t")
  
}

Put together all the diagnosis data

In [ ]:
for( i in 1:length( fileListDiag ) ){
  print(i)
  selection <- read.delim( paste0( "./phase1.1/latest/",fileListDiag[i]), sep = ",", colClasses = "character")
  if( grepl( "UNC", x=fileListDiag[i], fixed = TRUE) == TRUE ){
    selection[ ,4:7][ selection[, 4:7] == 10] <- -99 
  }
  colnames(selection) <- tolower( colnames( selection ) )
  obf <- obfuscation[ tolower(obfuscation$siteid) == tolower(selection$siteid[1]), ]
  if( i== 1){
    if( obf$obfuscation != "none"){
      selection[ selection == -99 ] <- 0.5 * as.numeric( obf$obfuscation )
    }
    allDiagnosis <- selection
  }else{ 
    if( obf$obfuscation != "none"){
      selection[ selection == -99 ] <- 0.5 * as.numeric( obf$obfuscation )
    }
    allDiagnosis <- rbind( allDiagnosis, selection )
    
  }
}
allDiagnosis <- allDiagnosis[, c("siteid", "icd_code_3chars", "icd_version", "num_patients_all_since_admission" )]
colnames( allDiagnosis)[4] <- paste0( colnames( allDiagnosis)[4], "_diagnosis")

Put together all the demographic data

In [ ]:
for( i in 1:length( fileListDemog ) ){
  print(i)
  selection <- read.delim( paste0( "./phase1.1/latest/",fileListDemog[i]), sep = ",", colClasses = "character")
  
  if( grepl( "UNC", x=fileListDemog[i], fixed = TRUE) == TRUE ){
    selection[ selection == 10] <- -99 
  }
  
  colnames( selection ) <- tolower( colnames( selection ))
  selection <- selection[ selection$sex == "all" &
                            selection$age_group == "all" & 
                            selection$race == "all", 
                          c("siteid", "sex", "age_group", "race", "num_patients_all") ]
  
  #some sites do not report the all, all, all, so we have to compute it
  if(nrow(selection)==0){
    selection <- read.delim( paste0( "./phase1.1/latest/",fileListDemog[i]), sep = ",", colClasses = "character")
    if( grepl( "UNC", x=fileListDemog[i], fixed = TRUE) == TRUE ){
      selection[ selection == 10] <- -99 
    }
    colnames( selection ) <- tolower( colnames( selection ))
    selection <- selection[ selection$age_group == "all" & 
                              selection$race == "all", 
                            c("siteid", "sex", "age_group", "race", "num_patients_all") ]
    nr <- c(selection$siteid[1], "all", "all", "all", 
            sum(as.numeric(selection$num_patients_all)))
    selection <- rbind( selection, nr)
    selection <- selection[ selection$sex == "all" &
                              selection$age_group == "all" & 
                              selection$race == "all", ]
  }
  obf <- obfuscation[ tolower(obfuscation$siteid) == tolower(selection$siteid[1]), ]
  
  if( i== 1){
    if( obf$obfuscation != "none"){
      selection[ selection == -99 ] <- 0.5 * as.numeric( obf$obfuscation )
    }
    allDemographics <- selection
  }else{ 
    if( obf$obfuscation != "none"){
      selection[ selection == -99 ] <- 0.5 * as.numeric( obf$obfuscation )
    }
    allDemographics <- rbind( allDemographics, selection )
  }
}
allDemographics<- allDemographics[, c("siteid", "num_patients_all")]
colnames(allDemographics) <- c("siteid", "totalPatients")

Put all the information together merging all dataframes by site id and transform unknown values (-999) in NAs.

In [ ]:
finalDataSet <- merge( allDiagnosis, allDemographics, by = "siteid")

#transform -999 in NA
finalDataSet[,c(4:5)] <- sapply(finalDataSet[,c(4:5)],as.numeric)
finalDataSet[ finalDataSet == -999 ] <- NA

We determine a subset of ICD codes we would like to review.

In [ ]:
codesToReview <- c("J96","R00","J12","R57","J90","N17" )

### Summary of diagnosis counts per site

In [ ]:
bySite <- aggregateCounts( inputData = finalDataSet, aggregationType = "Site")

bySite <- bySite[ order( bySite$perc_patients_since_adimission_diagnosis, decreasing = TRUE), ]


bySiteSelection <- aggregateCounts( inputData = finalDataSet, 
                                    aggregationType = "Site", 
                                    codeSelection = codesToReview)


bySiteSelection <- bySiteSelection[ order( bySiteSelection$perc_patients_since_adimission_diagnosis, 
                                    decreasing = TRUE), ]

#write the output
write.table(bySite, file="./siteOutput_combined.txt", 
            col.names = TRUE, row.names = FALSE, sep = "\t", quote = FALSE)
write.table(bySiteSelection, file="./siteSubsetOutput.txt", 
            col.names = TRUE, row.names = FALSE, sep = "\t", quote = FALSE)

### Summary of diagnosis counts per country

In [ ]:
byCountry <- aggregateCounts( inputData = finalDataSet, aggregationType = "Country")

byCountrySelection <- aggregateCounts( inputData = finalDataSet, 
                                       aggregationType = "Country", 
                                       codeSelection = codesToReview)

countries <- c("USA", "France", "UK", "Spain", "Singapore", "Germany")
countryOutputSl <- excelTableFormat( input = byCountrySelection, 
                                     countries = countries)


countryOutput <- excelTableFormat( input = byCountry, 
                                   countries = countries)
                                   
write.table(byCountry, file="./countryOutput.txt", 
            col.names = TRUE, row.names = FALSE, sep = "\t", quote = FALSE)
write.table(byCountrySelection, file="./countrySubsetOutput.txt", 
            col.names = TRUE, row.names = FALSE, sep = "\t", quote = FALSE)

### Summary of diagnosis counts all

In [ ]:
byAll <- aggregateCounts( inputData = finalDataSet, aggregationType = "All")

byAllSelection <- aggregateCounts( inputData = finalDataSet, 
                                   aggregationType = "All", 
                                   codeSelection = codesToReview)
                                   
write.table(byAllSelection, file="./allSelectionOutput.txt", 
            col.names = TRUE, row.names = FALSE, sep = "\t", quote = FALSE)
write.table(byAll, file="./allOutput.txt", 
            col.names = TRUE, row.names = FALSE, sep = "\t", quote = FALSE)